In [2]:
# === ADIM 1: CIC-DDoS 2019 dataset birleştirme ve temizleme ===

import os, glob, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

DATA_DIR = "/kaggle/input/cicddos2019"
TARGET   = "Label"

def extract_label_from_filename(fname):
    """
    Örn: 'Syn-training.parquet' → 'Syn'
    """
    return os.path.basename(fname).split("-")[0]

# 1) Tüm parquet dosyalarını yükle
frames = []
for path in sorted(glob.glob(os.path.join(DATA_DIR, "*.parquet"))):
    df = pd.read_parquet(path)
    
    # Etiket ekle
    df[TARGET] = extract_label_from_filename(path)
    
    frames.append(df)

# 2) Tüm veriyi tek dataframe’de birleştir
full_df = pd.concat(frames, ignore_index=True)
print("Ham veri boyutu:", full_df.shape)

# 3) Inf / NaN temizle
full_df = full_df.replace([np.inf, -np.inf], np.nan)
full_df = full_df.dropna(axis=0)   # çok büyük dataset olduğundan satır silmek güvenli
print("NaN temizliği sonrası:", full_df.shape)

# 4) Gereksiz kolonlar (Flow ID, Timestamp, vs.)
drop_cols = [c for c in full_df.columns 
             if "Flow ID" in c or "Timestamp" in c or "Src IP" in c or "Dst IP" in c]

if drop_cols:
    full_df = full_df.drop(columns=drop_cols)
    print("Gereksiz ID/IP kolonları silindi:", drop_cols)

# 5) Numeric olmayan kolonları at
numeric_cols = [c for c in full_df.columns if pd.api.types.is_numeric_dtype(full_df[c])]
non_numeric  = [c for c in full_df.columns if c not in numeric_cols + [TARGET]]

if non_numeric:
    full_df = full_df.drop(columns=non_numeric)
    print("Sayısal olmayan kolonlar silindi:", non_numeric)

# 6) Sabit (nunique=1) kolonları at
nunique = full_df[numeric_cols].nunique()
const_cols = nunique[nunique <= 1].index.tolist()

if const_cols:
    full_df = full_df.drop(columns=const_cols)
    print("Sabit kolonlar silindi:", const_cols)

print("\n📌 ADIM 1 TAMAMLANDI — TEMİZ DATA")
print("Final boyut:", full_df.shape)
print("Sınıflar:", full_df[TARGET].unique())


Ham veri boyutu: (431371, 78)
NaN temizliği sonrası: (431371, 78)
Sabit kolonlar silindi: ['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'FIN Flag Count', 'PSH Flag Count', 'ECE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']

📌 ADIM 1 TAMAMLANDI — TEMİZ DATA
Final boyut: (431371, 66)
Sınıflar: ['DNS' 'LDAP' 'MSSQL' 'NTP' 'NetBIOS' 'Portmap' 'SNMP' 'Syn' 'TFTP' 'UDP'
 'UDPLag']


Tüm CIC-DDoS-2019 trafik dosyaları birleşik bir veri setinde toplandı ve etiketler dosya adından otomatik olarak üretildi. Flow ID, Timestamp ve IP adresleri gibi saldırı tipiyle ilişkisi olmayan kolonlar çıkarıldı. Inf/NaN değerler temizlendi, sabit ve sayısal olmayan değişkenler veri setinden atıldı. Bu işlem sonucunda tamamen temizlenmiş ve analiz için hazır bir veri seti elde edilmiştir.

In [3]:
# === ADIM 2: Stratified Train/Test Split (%80 / %20) ===

from sklearn.model_selection import train_test_split

df_clean = full_df.copy()   # Adım 1'den gelen temiz dataset

X = df_clean.drop(columns=[TARGET])
y = df_clean[TARGET]

print("Tüm veri:", X.shape)

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    shuffle=True,
    stratify=y
)

print("Train:", X_train.shape, "| Test:", X_test.shape)
print("\nTrain sınıf dağılımı:\n", y_train.value_counts())
print("\nTest sınıf dağılımı:\n", y_test.value_counts())


Tüm veri: (431371, 65)
Train: (345096, 65) | Test: (86275, 65)

Train sınıf dağılımı:
 Label
NTP        107739
TFTP        97466
Syn         56994
UDP         24186
UDPLag      20083
MSSQL       15246
LDAP         7637
DNS          5362
Portmap      4084
SNMP         3214
NetBIOS      3085
Name: count, dtype: int64

Test sınıf dağılımı:
 Label
NTP        26935
TFTP       24367
Syn        14249
UDP         6046
UDPLag      5021
MSSQL       3811
LDAP        1909
DNS         1341
Portmap     1021
SNMP         804
NetBIOS      771
Name: count, dtype: int64


Tüm veri kümesi rastgele karıştırılmış ve sınıf oranlarının korunması amacıyla Stratified Train-Test Split yöntemiyle %80 eğitim ve %20 test olacak şekilde bölünmüştür. CIC-DDoS-2019 veri seti farklı günlerde toplanmış trafiklerden oluştuğu için orijinal dosya bazlı bölme yaklaşımı dağılım tutarsızlığı yaratmaktadır. Bu nedenle stratified split kullanımı sınıflandırma algoritmalarının optimizasyon etkisini daha doğru ölçmek için tercih edilmiştir.

In [4]:
# === ADIM 3: Custom SMOTE (imblearn olmadan) ===
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

# 1) Label Encode
enc = LabelEncoder()
y_train_enc = enc.fit_transform(y_train)
y_test_enc  = enc.transform(y_test)

print("Sınıf -> ID eşleşmesi:", dict(zip(enc.classes_, enc.transform(enc.classes_))))

# 2) Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 3) Custom SMOTE fonksiyonu
def custom_smote(X, y, k=5):
    X = np.array(X)
    y = np.array(y)

    classes, counts = np.unique(y, return_counts=True)
    max_count = counts.max()

    new_X = [X]
    new_y = [y]

    for cls, count in zip(classes, counts):
        if count == max_count:
            continue

        need = max_count - count
        cls_idx = np.where(y == cls)[0]
        X_cls = X[cls_idx]

        nn = NearestNeighbors(n_neighbors=min(k, len(X_cls)))
        nn.fit(X_cls)
        neigh_idx = nn.kneighbors(X_cls, return_distance=False)

        synthetic = []
        for i in range(need):
            row = np.random.choice(cls_idx)
            neigh = np.random.choice(neigh_idx[row % len(X_cls)])
            diff = X[neigh] - X[row]
            synthetic.append(X[row] + np.random.rand() * diff)

        new_X.append(np.array(synthetic))
        new_y.append(np.full(need, cls))

    X_out = np.vstack(new_X)
    y_out = np.concatenate(new_y)
    return X_out, y_out

# 4) SMOTE uygula
X_train_bal, y_train_bal = custom_smote(X_train_scaled, y_train_enc, k=5)

print("\nYeni Train boyutu:", X_train_bal.shape)
print("Yeni dağılım:", pd.Series(y_train_bal).value_counts())


Sınıf -> ID eşleşmesi: {'DNS': 0, 'LDAP': 1, 'MSSQL': 2, 'NTP': 3, 'NetBIOS': 4, 'Portmap': 5, 'SNMP': 6, 'Syn': 7, 'TFTP': 8, 'UDP': 9, 'UDPLag': 10}

Yeni Train boyutu: (1185129, 65)
Yeni dağılım: 3     107739
8     107739
2     107739
10    107739
9     107739
7     107739
1     107739
4     107739
5     107739
6     107739
0     107739
Name: count, dtype: int64


Etiketler, sınıflandırma algoritmalarında kullanılmak üzere tamsayı değerler ile kodlanmıştır (Label Encoding). Bu, çok sınıflı optimizasyon sürecini kolaylaştırmak için standart bir adımdır.Sayısal özellikler, standart sapma ve ortalama bazlı olarak ölçeklendirilmiş (StandardScaler) ve tüm özellikler aynı büyüklük mertebesine getirilmiştir. Bu, hem sınıflandırma algoritmalarının kararlılığını hem de meta-sezgisel optimizasyonun performansını artırır.

In [5]:
print("X_train_bal var mı?", 'X_train_bal' in globals())
print("y_train_bal var mı?", 'y_train_bal' in globals())


X_train_bal var mı? True
y_train_bal var mı? True


In [ ]:
# === ADIM 4 (Optimize): Baseline modellerin karşılaştırılması ===

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import pandas as pd
import numpy as np

RANDOM_STATE = 42

print("Train (balanced):", X_train_bal.shape)
print("Test (scaled):   ", X_test_scaled.shape)

# 🔥 SADELEŞTİRİLMİŞ MODELLER — HEPSİ VERİYE UYGUN VE HIZLI
models = {
    "NaiveBayes": GaussianNB(),

    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ),

    "LightGBM": LGBMClassifier(
        random_state=RANDOM_STATE,
        n_estimators=200,
        n_jobs=-1
    ),

    "XGBoost": XGBClassifier(
        random_state=RANDOM_STATE,
        n_estimators=200,
        n_jobs=-1,
        tree_method="hist",
        objective="multi:softmax",
        num_class=len(np.unique(y_train_bal)),
        eval_metric="mlogloss",
        verbosity=0,
    ),
}

# 3-fold CV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

baseline_results = []

print("\n✅ Baz modeller (3-fold CV, balanced data):\n")
for name, clf in models.items():
    acc_scores = cross_val_score(
        clf, X_train_bal, y_train_bal,
        cv=cv, scoring="accuracy"
    )
    f1_scores = cross_val_score(
        clf, X_train_bal, y_train_bal,
        cv=cv, scoring="f1_macro"
    )

    acc_mean = acc_scores.mean()
    f1_mean  = f1_scores.mean()

    baseline_results.append({
        "Model": name,
        "CV_Accuracy": acc_mean,
        "CV_MacroF1": f1_mean
    })

    print(f"{name:10s} | Acc={acc_mean:.4f} | F1-macro={f1_mean:.4f}")

baseline_df = pd.DataFrame(baseline_results).sort_values(
    "CV_Accuracy", ascending=False
).reset_index(drop=True)

print("\n📊 Baz modellerin CV sonuç özeti:")
print(baseline_df)

# Test aşaması — ilk 2 modeli testte değerlendirelim
top_models = baseline_df["Model"].head(2).tolist()

print("\n🧪 Test seti değerlendirmesi (ilk 2 model):\n")

for name in top_models:
    clf = models[name]
    clf.fit(X_train_bal, y_train_bal)
    y_pred_test = clf.predict(X_test_scaled)

    acc_test = accuracy_score(y_test_enc, y_pred_test)
    f1_test  = f1_score(y_test_enc, y_pred_test, average="macro")

    print(f"=== {name} ===")
    print(f"Test Accuracy : {acc_test:.4f}")
    print(f"Test Macro-F1 : {f1_test:.4f}")
    print("Sınıf bazlı rapor:")
    print(classification_report(y_test_enc, y_pred_test, digits=4))
    print("-" * 60)


Bu adımda, SMOTE ile dengelenmiş eğitim verisi kullanılarak dört farklı sınıflandırma algoritması (Naive Bayes, Random Forest, LightGBM ve XGBoost) 3 katlı çapraz doğrulama ile değerlendirilmiştir. Amaç, optimizasyon uygulanmadan önce modellerin temel (baseline) doğruluk ve Macro-F1 performanslarını belirlemek ve ilerleyen adımlarda meta-sezgisel optimizasyon teknikleri ile elde edilecek iyileşmeyi karşılaştırmak için referans değerler oluşturmaktır. Çapraz doğrulamanın ardından en yüksek performansı gösteren ilk iki model gerçek test kümesi üzerinde ayrıca değerlendirilmiştir.